In [12]:
import numpy as np
import pandas as pd
import tensorflow
from gensim.models.word2vec import Word2Vec
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Conv1D, MaxPool1D, GlobalMaxPool1D, Embedding, Activation
from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
import re
from nltk.corpus import stopwords
from nltk.stem.snowball import PorterStemmer
from sklearn import preprocessing
import gensim
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.models import Sequential
import nltk

In [13]:
def preprocess_text(sen):
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sen)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)
    
    stops = stopwords.words('english')
    #print(stops)
    porter = PorterStemmer()
    for word in sentence.split():
        if word in stops:
            sentence = sentence.replace(word, '')
        sentence = sentence.replace(word, porter.stem(word))
    return sentence.lower()

In [14]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('../content/drive/MyDrive/Reviews.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:

df = df[['Text','Score']]
df['Score'].isnull().sum()
df['Score'].isnull().sum()
df.drop_duplicates(subset=['Text','Score'],keep='first',inplace=True)

def set_sent(score):
    if score <= 3:
        return 0
    else:
        return 1

df['sentiment']=df['Score'].apply(set_sent)
df = df[['Text','sentiment']]

# Separate into three sentiment groups
df_0 = df[df['sentiment'] == 0]
df_1 = df[df['sentiment'] == 1]

n = 25000

df_0 = df_0.sample(n=n, random_state=42, replace=False)
df_1 = df_1.sample(n=n, random_state=42, replace=False)
# print("df_neg1: ", df_neg1)
# print("df_0: ", df_0)
# print("df_1: ", df_1)

sub_df = pd.concat([df_0, df_1], axis=0)
X = sub_df['Text']
y = sub_df['sentiment']

In [16]:
mes = []
for i in X:
    mes.append(i.split())
print(mes[:2])

[['Some', 'neighborhood', 'cat', 'uses', 'our', 'backyard', 'as', 'her', 'sandbox', 'every', 'night.', 'We', 'sprinkled', 'Critter', 'Ridder', 'heavily', 'around', 'the', "cat's", 'favorite', 'spots.', 'Not', 'only', 'did', 'the', 'cat', 'come', 'back,', 'she', 'pooped', 'on', 'one', 'of', 'the', 'treated', 'spots', 'again.'], ['Very', 'nice', 'packaging', 'presentation', 'and', 'very', 'nice', 'assortment', 'of', 'candy.', 'The', 'problem', 'with', 'the', 'candy', 'was', 'that', 'the', 'Violet', 'candy', 'that', 'was', 'included', 'in', 'the', 'box', 'gave', 'almost', 'all', 'of', 'the', 'other', 'candy', 'in', 'the', 'box', 'a', 'flowery', 'flavor.']]


In [17]:
w2v_model = Word2Vec(mes, vector_size=100, window=6, min_count=1, workers=16)
print(w2v_model)

Word2Vec<vocab=162214, vector_size=100, alpha=0.025>


In [18]:
# Preprocess the text data
import nltk
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))
def preprocess(text):
    text = text.lower()
    text = ''.join([word for word in text if word not in string.punctuation])
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

X_train, X_test, y_train, y_test = train_test_split (X, y, test_size=0.2)

X_train = X_train.apply(preprocess)
X_test = X_test.apply(preprocess)

max_length = 100
# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
X_train = pad_sequences(X_train, maxlen=max_length, padding='post')
X_test = pad_sequences(X_test, maxlen=max_length, padding='post')
vocab_size = len(tokenizer.word_index) + 1
# Create a weight matrix for the embedding layer
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [19]:
# Define the CNN model
max_length = 300
model = Sequential()
model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length, trainable=False))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(256, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10


ValueError: ignored